In [1]:
import pathlib
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import anndata
from pybedtools import BedTool

## Parameter

In [2]:
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/axis-mch_rms_results_collapsed.tsv'
dmr_prefix = 'DGmCH'

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'
delta_cutoff=0.3
dms_cutoff = 1

In [3]:
# Parameters
dmr_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L5/IT-L5_rms_results_collapsed.tsv"
dmr_prefix = "DGm3c"
black_list_path = "/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz"
delta_cutoff = 0.4
dms_cutoff = 1


## Read DMR

In [4]:
# get column and n_sample
dmr_column = pd.read_csv(dmr_path, sep='\t', nrows=0).columns
n_sample = dmr_column.size - 6
dmr_column = dmr_column.map(lambda i: i[18:] if 'methylation_level_' in i else i)
samples = dmr_column[6:]

dmr_region_infos = []
mc_rates = []
hypo_sig_dict = {sample: [] for sample in samples}
hyper_sig_dict = {sample: [] for sample in samples}

dmr_df = pd.read_csv(dmr_path, sep='\t', skiprows=1, names=dmr_column)
print(dmr_df.shape[0], 'dmr before filter')
dmr_df.index = dmr_df.index.map(lambda i: f'{dmr_prefix}_{i}')

# save raw info
dmr_bed = dmr_df.iloc[:, :4].copy()
mc_rate = dmr_df.iloc[:, 6:].copy()

552735 dmr before filter


## Save unfiltered dmr info

In [5]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    hdf['Rate'] = mc_rate
    hdf['Bed'] = dmr_bed

In [6]:
dmr_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(pathlib.Path(dmr_path).parent / 'TotalDMR.nofilter.bed', 
                                                   index=None, header=None, sep='\t')

## Filter DMR

In [7]:
# filter by delta
delta_judge = (mc_rate.max(axis=1) - mc_rate.min(axis=1)) > delta_cutoff
delta_index = mc_rate.index[delta_judge]

print(delta_index.size, 'dmr pass delta filter')

207109 dmr pass delta filter


In [8]:
# filter by blacklist
black_list = pd.read_csv(black_list_path, sep='\t', header=None)
black_list = BedTool.from_dataframe(black_list)

_dmr_bed = BedTool.from_dataframe(dmr_bed.reset_index().iloc[:, [1,2,3,0]])
filtered_dmr_bed = _dmr_bed.intersect(black_list, v=True).to_dataframe().set_index('name')

white_index = filtered_dmr_bed.index

print(white_index.size, 'dmr pass blacklist filter')

546728 dmr pass blacklist filter


In [9]:
dmr_df = dmr_df.loc[delta_index & white_index]
dmr_df = dmr_df[dmr_df['number_of_dms'] >= dms_cutoff]
print(dmr_df.shape[0], 'dmr after filter')
dmr_df.head()

204828 dmr after filter


,#chr,start,end,number_of_dms,hypermethylated_samples,hypomethylated_samples,IT-L5+SSp,IT-L5+PFC,IT-L5+AI,IT-L5+MOs,IT-L5+MOp,IT-L5+ORB,IT-L5+SSs,IT-L5+ACA
DGm3c_0,chr1,3003641,3003641,1,NaN,NaN,0.608696,0.222222,0.600000,0.225806,0.166667,NaN,0.571429,0.372549
DGm3c_5,chr1,3095658,3095658,1,"IT-L5+PFC,IT-L5+AI",IT-L5+MOp,0.342105,0.710145,0.818182,0.365854,0.250000,0.600000,0.392857,0.511905
DGm3c_7,chr1,3108226,3108226,1,"IT-L5+PFC,IT-L5+AI",IT-L5+SSp,0.050000,0.396825,0.523810,0.150000,0.070423,0.241379,0.085714,0.184615
DGm3c_9,chr1,3114936,3115276,4,"IT-L5+SSp,IT-L5+ACA,IT-L5+MOp","IT-L5+PFC,IT-L5+MOs,IT-L5+AI",0.870588,0.544776,0.413793,0.560000,0.773529,0.593220,0.761062,0.741667
DGm3c_10,chr1,3121820,3121820,1,"IT-L5+PFC,IT-L5+ACA",IT-L5+SSp,0.257143,0.672727,0.480000,0.464286,0.348485,0.423077,0.260870,0.637500


## Assign DMR to each sample

In [10]:
# parse results
for sample in samples:
    # here saved a list of np.array
    hypo_index = dmr_df[dmr_df['hypomethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hyper_index = dmr_df[dmr_df['hypermethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hypo_sig_dict[sample] = hypo_index
    hyper_sig_dict[sample] = hyper_index
    print(sample, 'HypoDMR ', hypo_index.size, sep='\t')
    print(sample, 'HyperDMR', hyper_index.size, sep='\t')


IT-L5+SSp	HypoDMR 	68184
IT-L5+SSp	HyperDMR	41504
IT-L5+PFC	HypoDMR 	30794
IT-L5+PFC	HyperDMR	86126
IT-L5+AI	HypoDMR 	33931
IT-L5+AI	HyperDMR	36573
IT-L5+MOs	HypoDMR 	51106
IT-L5+MOs	HyperDMR	13727
IT-L5+MOp	HypoDMR 	63615
IT-L5+MOp	HyperDMR	9278
IT-L5+ORB	HypoDMR 	9100
IT-L5+ORB	HyperDMR	16944
IT-L5+SSs	HypoDMR 	37760
IT-L5+SSs	HyperDMR	18470
IT-L5+ACA	HypoDMR 	29877
IT-L5+ACA	HyperDMR	93582


In [11]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    for sample, hypo_index in hypo_sig_dict.items():
        hdf[f'HypoDMR/{sample}'] = pd.Series(hypo_index)
    for sample, hyper_index in hyper_sig_dict.items():
        hdf[f'HyperDMR/{sample}'] = pd.Series(hyper_index)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L5+SSp'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L5+PFC'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L5+AI'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming

## Dump DMR bed

In [12]:
hypo_dir = pathlib.Path(dmr_path).parent / 'HypoDMR'
hypo_dir.mkdir(exist_ok=True)
for sample, hypo_index in hypo_sig_dict.items():
    _bed = dmr_bed.loc[hypo_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hypo_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

In [13]:
hyper_dir = pathlib.Path(dmr_path).parent / 'HyperDMR'
hyper_dir.mkdir(exist_ok=True)
for sample, hyper_index in hyper_sig_dict.items():
    _bed = dmr_bed.loc[hyper_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hyper_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

## DMR hits matrix

In [14]:
sig_dict = hypo_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hypo_dir / 'TotalHits.h5ad')


... storing '#chr' as categorical


In [15]:
sig_dict = hyper_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hyper_dir / 'TotalHits.h5ad')

... storing '#chr' as categorical
